In [1]:
1+2


3

Haha,got it work

# Why don't we start the project NAS?

## First we need to find out how do the first paper build the Architecture and train datasets

In [ ]:
#---------------------------------- Get familiar with the code in autokeras-----------------------------#
# ----------------classifier.py
# init->fit->evaluate

#fit->
#    # Transform x,y_train.
#    # Create the searcher and save on disk  
#    # Divide training data into training and testing data
#    # LOOP
       $$$run_searcher_once(train_data, test_data, self.path)$$$
         if len(self.load_searcher().history) >= Constant.MAX_MODEL_NUM#1000:
                break
#    #LOOP END
#
#
#         # 
#         ###run_searcher_once()->autokeras.utils.pickle_from_file() CALL searcher.search()
#             input_shape = x_train.shape[1:]
            n_classes = self.y_encoder.n_classes #An integer, the number of classes
            self.searcher_args['n_classes'] = n_classes #An integer, the number of classes
            self.searcher_args['input_shape'] = input_shape #A tuple. e.g. (28, 28, 1)
            self.searcher_args['path'] = self.path#The path to the directory to save the searcher
            self.searcher_args['verbose'] = self.verbose#A boolean. Whether to output the intermediate information to stdout.
            $$$$  searcher = BayesianSearcher(**self.searcher_args) $$$$
            self.save_searcher(searcher)
            self.searcher = True
#-----------------Search.py

#           ->__init__()
                 default_model_len=Constant.MODEL_LEN,#3
                 default_model_width=Constant.MODEL_WIDTH,#64
                 beta=Constant.BETA,#0.576
                 kernel_lambda=Constant.KERNEL_LAMBDA#1.0
                if trainer_args is None:
                    trainer_args = {}
                if 'max_iter_num' not in self.trainer_args:
                self.trainer_args['max_iter_num'] = Constant.SEARCH_MAX_ITER#200
                self.gpr = IncrementalGaussianProcess(kernel_lambda)
                self.search_tree = SearchTree()
                self.training_queue = []
                self.x_queue = []
                self.y_queue = []
                self.beta = beta
                if t_min is None:
                    t_min = Constant.T_MIN#0.0001
                self.t_min = t_min
#           ->search()
#             ->init_search()  start
#------------------generator.py
                graph = DefaultClassifierGenerator(self.n_classes,
                self.input_shape).generate(self.default_model_len,
                                          self.default_model_width)
                    $$$$$generate$$$$$
                        model_width=Constant.MODEL_WIDTH#64):
                        pooling_len = int(model_len#3 / 4)
                        graph = Graph(self.input_shape, False)
#------------------Graph.py                                          
                          $$$$$$Graph.__init__start$$$$$$
                                    self.weighted = weighted#A boolean of whether the weights and biases in the neural network should be included in the graph.
                                    self.node_list = []#A list of integers, the indices of the list are the identifiers.
                                    self.layer_list = []#A list of stub layers, the indices of the list are the identifiers.
                                    # node id start with 0
                                    self.node_to_id = {}#A dict instance mapping from node integers to their identifiers.
                                    self.layer_to_id = {}# A dict instance mapping from stub layers to their identifiers.
                                    self.layer_id_to_input_node_ids = {}#A dict instance mapping from layer identifiers to their input nodes identifiers.
                                    self.layer_id_to_output_node_ids = {}
                                    self.adj_list = {}#A two dimensional list. The adjacency list of the graph. The first dimension is identified by tensor identifiers. In each edge list, the elements are two-element tuples of (tensor identifier, layer identifier).
                                    self.reverse_adj_list = {}#A reverse adjacent list in the same format as adj_list.
                                    self.operation_history = []#A list saving all the network morphism operations.

                                    self.vis = None#A dictionary of temporary storage for whether an local operation has been done during the network morphism.
                                    self._add_node(Node(input_shape))
                          $$$$$$Graph.__init__end$$$$$$
                        temp_input_channel = self.input_shape[-1]
                        output_node_id = 0
                        for i in range(model_len):
                            output_node_id = graph.add_layer(StubReLU(), output_node_id)
                            output_node_id = graph.add_layer(StubConv(temp_input_channel, model_width, kernel_size=3), output_node_id)
                            output_node_id = graph.add_layer(StubBatchNormalization(model_width), output_node_id)
                            output_node_id = graph.add_layer(StubDropout(Constant.CONV_DROPOUT_RATE), output_node_id)
                            temp_input_channel = model_width
                            if pooling_len == 0 or ((i + 1) % pooling_len == 0 and i != model_len - 1):
                                output_node_id = graph.add_layer(StubPooling(), output_node_id)

                        output_node_id = graph.add_layer(StubFlatten(), output_node_id)
                        output_node_id = graph.add_layer(StubDense(graph.node_list[output_node_id].shape[0], self.n_classes),
                                                         output_node_id)
                        graph.add_layer(StubSoftmax(), output_node_id)
                        return graph
                model_id = self.model_count
                self.model_count += 1
                self.training_queue.append((graph, -1, model_id))
                self.descriptors.append(graph.extract_descriptor())
                for child_graph in default_transform(graph):
                    child_id = self.model_count
                    self.model_count += 1
                    self.training_queue.append((child_graph, model_id, child_id))
                    self.descriptors.append(child_graph.extract_descriptor())
                if self.verbose:
                    print('Initialization finished.') 
#            ->init_search() end
#            # Start the new process for training.
            graph, father_id, model_id = self.training_queue.pop(0)
            if self.verbose:
                print('Training model ', model_id)
                multiprocessing.set_start_method('spawn', force=True)
                pool = multiprocessing.Pool(1)
                $$$$$$$train_results = pool.map_async(train, [(graph, train_data, test_data, self.trainer_args,
                                                        os.path.join(self.path, str(model_id) + '.png'), self.verbose)])                        
#            # Do the search in current thread.
            if not self.training_queue:
                new_graph, new_father_id = self.maximize_acq()# produce the next graph based on the
                new_model_id = self.model_count
                self.model_count += 1
                self.training_queue.append((new_graph, new_father_id, new_model_id))
                descriptor = new_graph.extract_descriptor()
                self.descriptors.append(new_graph.extract_descriptor())

            accuracy, loss, graph = train_results.get()[0]
            pool.terminate()
            pool.join()
            self.add_model(accuracy, loss, graph, model_id)
            self.search_tree.add_child(father_id, model_id)
            self.gpr.fit(self.x_queue, self.y_queue)
            self.x_queue = []
            self.y_queue = []

            pickle_to_file(self, os.path.join(self.path, 'searcher'))
            self.export_json(os.path.join(self.path, 'history.json'))


# ->DefaultClassifierGenerator->Graph


### Minst.py

In [ ]:
### Minst dataset using autokeras

In [1]:
from keras.datasets import mnist
from autokeras.classifier import ImageClassifier

if __name__ == '__main__':
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(x_train.shape + (1,))
    x_test = x_test.reshape(x_test.shape + (1,))

    clf = ImageClassifier(verbose=True, augment=False)
    clf.fit(x_train, y_train, time_limit=9 * 60 * 60)
    clf.final_fit(x_train, y_train, x_test, y_test, retrain=True)
    y = clf.evaluate(x_test, y_test)
    print(y * 100)

Using TensorFlow backend.


Initializing search.
Plotting model init  1111111111
Initialization finished.
before the new proc
Plotting model  0
Training model  0
Saving model.
Model ID: 0
Loss: tensor(1.1282)
Accuracy 99.18
before the new proc
Plotting model  1
Training model  1
Father ID:  0
[('to_wider_model', 6, 64)]
Saving model.
Model ID: 1
Loss: tensor(0.9911)
Accuracy 99.396
before the new proc
Plotting model  2
Training model  2
Father ID:  1
[('to_wider_model', 11, 64)]
Saving model.
Model ID: 2
Loss: tensor(1.2862)
Accuracy 99.212
before the new proc
Plotting model  3
Training model  3
Father ID:  2
[('to_wider_model', 6, 128)]
Saving model.
Model ID: 3
Loss: tensor(1.0435)
Accuracy 99.41599999999998
...............................................
Epoch 1: loss 2.6209282875061035, accuracy 98.87
...............................................
Epoch 2: loss 2.1810882091522217, accuracy 99.15
...............................................
Epoch 3: loss 1.8129054307937622, accuracy 99.14
.................

### How to visualization

#### code alike

In [10]:
# check keras status
import keras
from keras.utils.vis_utils import plot_model
# check new plot func 
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot



#    plot_model(model,
               to_file='model.png',
               show_shapes=False,
               show_layer_names=True,
               rankdir='TB')
#plot(model, to_file='model.png')                    
    """Converts a Keras model to dot format and save to a file.

    # Arguments
        model: A Keras model instance
        to_file: File name of the plot image.
        show_shapes: whether to display shape information.
        show_layer_names: whether to display layer names.
        rankdir: `rankdir` argument passed to PyDot,
            a string specifying the format of the plot:
            'TB' creates a vertical plot;
            'LR' creates a horizontal plot.
    """

## Mapping from code to paper

In [ ]:
#aqusition function
    def maximize_acq(self):
        model_ids = self.search_tree.adj_list.keys()
        target_graph = None
        father_id = None
        descriptors = deepcopy(self.descriptors)

        # Initialize the priority queue.
        pq = PriorityQueue()
        temp_list = []
        for model_id in model_ids:
            accuracy = self.get_accuracy_by_id(model_id)
            temp_list.append((accuracy, model_id))
        temp_list = sorted(temp_list)
        for accuracy, model_id in temp_list:
            graph = self.load_model_by_id(model_id)
            graph.clear_operation_history()
            pq.put(Elem(accuracy, model_id, graph))

        t = 1.0
        t_min = self.t_min
        alpha = 0.9
        max_acq = -1
        while not pq.empty() and t > t_min:
            elem = pq.get()
            temp_exp = min((elem.accuracy - max_acq) / t, 709.0)
            ap = math.exp(temp_exp)
            if ap > random.uniform(0, 1):
                graphs = transform(elem.graph)

                for temp_graph in graphs:
                    if contain(descriptors, temp_graph.extract_descriptor()):
                        continue

                    temp_acq_value = self.acq(temp_graph)
                    pq.put(Elem(temp_acq_value, elem.father_id, temp_graph))
                    descriptors.append(temp_graph.extract_descriptor())
                    if temp_acq_value > max_acq:
                        max_acq = temp_acq_value
                        father_id = elem.father_id
                        target_graph = temp_graph
            t *= alpha

        nm_graph = self.load_model_by_id(father_id)
        if self.verbose:
            print('Father ID: ', father_id)
            print(target_graph.operation_history)
        for args in target_graph.operation_history:
            getattr(nm_graph, args[0])(*list(args[1:]))
        return nm_graph, father_id

    def acq(self, graph):
        mean, std = self.gpr.predict(np.array([graph.extract_descriptor()]))
        return mean + self.beta * std
#kernel
def layer_distance(a, b):
    return abs(a - b) * 1.0 / max(a, b)


def layers_distance(list_a, list_b):
    len_a = len(list_a)
    len_b = len(list_b)
    f = np.zeros((len_a + 1, len_b + 1))
    f[-1][-1] = 0
    for i in range(-1, len_a):
        f[i][-1] = i + 1
    for j in range(-1, len_b):
        f[-1][j] = j + 1
    for i in range(len_a):
        for j in range(len_b):
            f[i][j] = min(f[i][j - 1] + 1, f[i - 1][j] + 1, f[i - 1][j - 1] + layer_distance(list_a[i], list_b[j]))
    return f[len_a - 1][len_b - 1]


def skip_connection_distance(a, b):
    if a[2] != b[2]:
        return 1.0
    len_a = abs(a[1] - a[0])
    len_b = abs(b[1] - b[0])
    return (abs(a[0] - b[0]) + abs(len_a - len_b)) / (max(a[0], b[0]) + max(len_a, len_b))


def skip_connections_distance(list_a, list_b):
    distance_matrix = np.zeros((len(list_a), len(list_b)))
    for i, a in enumerate(list_a):
        for j, b in enumerate(list_b):
            distance_matrix[i][j] = skip_connection_distance(a, b)
    return distance_matrix[linear_sum_assignment(distance_matrix)].sum() + abs(len(list_a) - len(list_b))


def edit_distance(x, y, kernel_lambda):
    ret = 0
    ret += layers_distance(x.conv_widths, y.conv_widths)
    ret += layers_distance(x.dense_widths, y.dense_widths)
    ret += kernel_lambda * skip_connections_distance(x.skip_connections, y.skip_connections)
    return ret


    def incremental_fit(self, train_x, train_y):
        if not self._first_fitted:
            raise ValueError("The first_fit function needs to be called first.")

        train_x, train_y = np.array(train_x), np.array(train_y)

        # Incrementally compute K
        up_right_k = self.edit_distance_matrix(self.kernel_lambda, self._x, train_x)  # Shape (len(X_train_), len(train_x))
        down_left_k = np.transpose(up_right_k)
        down_right_k = self.edit_distance_matrix(self.kernel_lambda, train_x)
        up_k = np.concatenate((self._distance_matrix, up_right_k), axis=1)
        down_k = np.concatenate((down_left_k, down_right_k), axis=1)
        self._distance_matrix = np.concatenate((up_k, down_k), axis=0)
        self._distance_matrix = bourgain_embedding_matrix(self._distance_matrix)
        self._k_matrix = 1.0 / np.exp(self._distance_matrix)
        diag = np.diag_indices_from(self._k_matrix)
        diag = (diag[0][-len(train_x):], diag[1][-len(train_x):])
        self._k_matrix[diag] += self.alpha

        self._x = np.concatenate((self._x, train_x), axis=0)
        self._y = np.concatenate((self._y, train_y), axis=0)

        self._l_matrix = cholesky(self._k_matrix, lower=True)  # Line 2

        self._alpha_vector = cho_solve((self._l_matrix, True), self._y)  # Line 3

        return self

    @property
    def first_fitted(self):
        return self._first_fitted

    def first_fit(self, train_x, train_y):
        train_x, train_y = np.array(train_x), np.array(train_y)

        self._x = np.copy(train_x)
        self._y = np.copy(train_y)

        self._distance_matrix = self.edit_distance_matrix(self.kernel_lambda, self._x)
        self._distance_matrix = bourgain_embedding_matrix(self._distance_matrix)
        self._k_matrix = 1.0 / np.exp(self._distance_matrix)
        self._k_matrix[np.diag_indices_from(self._k_matrix)] += self.alpha

        self._l_matrix = cholesky(self._k_matrix, lower=True)  # Line 2

        self._alpha_vector = cho_solve((self._l_matrix, True), self._y)  # Line 3

        self._first_fitted = True
        return self
def edit_distance_matrix(kernel_lambda, train_x, train_y=None):
    if train_y is None:
        ret = np.zeros((train_x.shape[0], train_x.shape[0]))
        for x_index, x in enumerate(train_x):
            for y_index, y in enumerate(train_x):
                if x_index == y_index:
                    ret[x_index][y_index] = 0
                elif x_index < y_index:
                    ret[x_index][y_index] = edit_distance(x, y, kernel_lambda)
                else:
                    ret[x_index][y_index] = ret[y_index][x_index]
        return ret
    ret = np.zeros((train_x.shape[0], train_y.shape[0]))
    for x_index, x in enumerate(train_x):
        for y_index, y in enumerate(train_y):
            ret[x_index][y_index] = edit_distance(x, y, kernel_lambda)
    return ret


def vector_distance(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.linalg.norm(a - b)


def bourgain_embedding_matrix(distance_matrix):
    distance_matrix = np.array(distance_matrix)
    n = len(distance_matrix)
    if n == 1:
        return distance_matrix
    np.random.seed(123)
    distort_elements = []
    r = range(n)
    k = int(math.ceil(math.log(n) / math.log(2) - 1))
    t = int(math.ceil(math.log(n)))
    counter = 0
    for i in range(0, k + 1):
        for t in range(t):
            s = np.random.choice(r, 2 ** i)
            for j in r:
                d = min([distance_matrix[j][s] for s in s])
                counter += len(s)
                if i == 0 and t == 0:
                    distort_elements.append([d])
                else:
                    distort_elements[j].append(d)
    distort_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            distort_matrix[i][j] = distort_matrix[j][i] = vector_distance(distort_elements[i], distort_elements[j])
    return np.array(distort_matrix)

## Morph operator

In [ ]:
# graph morphism
def transform(graph):
    graphs = []
    for i in range(Constant.N_NEIGHBOURS):
        a = randrange(3)
        if a == 0:
            graphs.append(to_deeper_graph(deepcopy(graph)))
        elif a == 1:
            graphs.append(to_wider_graph(deepcopy(graph)))
        elif a == 2:
            graphs.append(to_skip_connection_graph(deepcopy(graph)))
    return list(filter(lambda x: legal_graph(x), graphs))


def to_wider_graph(graph):
    weighted_layer_ids = graph.wide_layer_ids()
    if len(weighted_layer_ids) <= 1:
        target_id = weighted_layer_ids[0]
    else:
        target_id = weighted_layer_ids[randint(0, len(weighted_layer_ids) - 1)]

    if is_layer(graph.layer_list[target_id], 'Conv'):
        n_add = graph.layer_list[target_id].filters
    else:
        n_add = graph.layer_list[target_id].units

    graph.to_wider_model(target_id, n_add)
    return graph


def to_skip_connection_graph(graph):
    # The last conv layer cannot be widen since wider operator cannot be done over the two sides of flatten.
    weighted_layer_ids = graph.skip_connection_layer_ids()
    index_a = randint(0, len(weighted_layer_ids) - 1)
    index_b = randint(0, len(weighted_layer_ids) - 1)
    if index_a == index_b:
        if index_b == 0:
            index_a = index_b + 1
        else:
            index_a = index_b - 1
    if index_a > index_b:
        index_a, index_b = index_b, index_a
    a_id = weighted_layer_ids[index_a]
    b_id = weighted_layer_ids[index_b]
    if graph.layer_list[a_id].filters == graph.layer_list[b_id].filters:
        graph.to_add_skip_model(a_id, b_id)
    else:
        graph.to_concat_skip_model(a_id, b_id)
    return graph


def to_deeper_graph(graph):
    weighted_layer_ids = graph.deep_layer_ids()
    target_id = weighted_layer_ids[randint(0, len(weighted_layer_ids) - 1)]
    if is_layer(graph.layer_list[target_id], 'Conv'):
        graph.to_conv_deeper_model(target_id, randint(1, 2) * 2 + 1)
    else:
        graph.to_dense_deeper_model(target_id)
    return graph

#graph func
    def to_dense_deeper_model(self, target_id):
        """Insert a dense layer after the target layer.

        Args:
            target_id: The ID of a dense layer.
        """
        self.operation_history.append(('to_dense_deeper_model', target_id))
        target = self.layer_list[target_id]
        new_layers = dense_to_deeper_block(target, self.weighted)
        output_id = self._dense_block_end_node(target_id)

        self._insert_new_layers(new_layers, output_id)
    def to_conv_deeper_model(self, target_id, kernel_size):
        """Insert a relu-conv-bn block after the target block.

        Args:
            target_id: A convolutional layer ID. The new block should be inserted after the block.
            kernel_size: An integer. The kernel size of the new convolutional layer.
        """
        self.operation_history.append(('to_conv_deeper_model', target_id, kernel_size))
        target = self.layer_list[target_id]
        new_layers = deeper_conv_block(target, kernel_size, self.weighted)
        output_id = self._conv_block_end_node(target_id)

        self._insert_new_layers(new_layers, output_id)

    def to_wider_model(self, pre_layer_id, n_add):
        """Widen the last dimension of the output of the pre_layer.

        Args:
            pre_layer_id: The ID of a convolutional layer or dense layer.
            n_add: The number of dimensions to add.
        """
        self.operation_history.append(('to_wider_model', pre_layer_id, n_add))
        pre_layer = self.layer_list[pre_layer_id]
        output_id = self.layer_id_to_output_node_ids[pre_layer_id][0]
        dim = layer_width(pre_layer)
        self.vis = {}
        self._search(output_id, dim, dim, n_add)
        for u in self.topological_order:
            for v, layer_id in self.adj_list[u]:
                self.node_list[v].shape = self.layer_list[layer_id].output_shape
    def to_concat_skip_model(self, start_id, end_id):
        """Add a weighted add concatenate connection from after start node to end node.

        Args:
            start_id: The convolutional layer ID, after which to start the skip-connection.
            end_id: The convolutional layer ID, after which to end the skip-connection.
        """
        self.operation_history.append(('to_concat_skip_model', start_id, end_id))
        conv_block_input_id = self._conv_block_end_node(start_id)
        conv_block_input_id = self.adj_list[conv_block_input_id][0][0]

        block_last_layer_input_id = self._conv_block_end_node(end_id)

        # Add the pooling layer chain.
        pooling_layer_list = self._get_pooling_layers(conv_block_input_id, block_last_layer_input_id)
        skip_output_id = conv_block_input_id
        for index, layer_id in enumerate(pooling_layer_list):
            skip_output_id = self.add_layer(deepcopy(self.layer_list[layer_id]), skip_output_id)

        block_last_layer_output_id = self.adj_list[block_last_layer_input_id][0][0]
        concat_input_node_id = self._add_node(deepcopy(self.node_list[block_last_layer_output_id]))
        self._redirect_edge(block_last_layer_input_id, block_last_layer_output_id, concat_input_node_id)

        concat_layer = StubConcatenate()
        concat_layer.input = [self.node_list[concat_input_node_id], self.node_list[skip_output_id]]
        concat_output_node_id = self._add_node(Node(concat_layer.output_shape))
        self._add_edge(concat_layer, concat_input_node_id, concat_output_node_id)
        self._add_edge(concat_layer, skip_output_id, concat_output_node_id)
        concat_layer.output = self.node_list[concat_output_node_id]
        self.node_list[concat_output_node_id].shape = concat_layer.output_shape

        # Add the concatenate layer.
        new_relu_layer = StubReLU()
        concat_output_node_id = self.add_layer(new_relu_layer, concat_output_node_id)
        new_conv_layer = StubConv(self.layer_list[start_id].filters + self.layer_list[end_id].filters,
                                  self.layer_list[end_id].filters, 1)
        concat_output_node_id = self.add_layer(new_conv_layer, concat_output_node_id)
        new_bn_layer = StubBatchNormalization(self.layer_list[end_id].filters)

        self._add_edge(new_bn_layer, concat_output_node_id, block_last_layer_output_id)
        new_bn_layer.input = self.node_list[concat_output_node_id]
        new_bn_layer.output = self.node_list[block_last_layer_output_id]
        self.node_list[block_last_layer_output_id].shape = new_bn_layer.output_shape

        if self.weighted:
            filters_end = self.layer_list[end_id].filters
            filters_start = self.layer_list[start_id].filters
            filter_shape = (1,) * (len(self.layer_list[end_id].get_weights()[0].shape) - 2)
            weights = np.zeros((filters_end, filters_end) + filter_shape)
            for i in range(filters_end):
                filter_weight = np.zeros((filters_end,) + filter_shape)
                filter_weight[(i, 0, 0)] = 1
                weights[i, ...] = filter_weight
            weights = np.concatenate((weights,
                                      np.zeros((filters_end, filters_start) + filter_shape)), axis=1)
            bias = np.zeros(filters_end)
            new_conv_layer.set_weights((add_noise(weights, np.array([0, 1])), add_noise(bias, np.array([0, 1]))))

            n_filters = filters_end
            new_weights = [add_noise(np.ones(n_filters, dtype=np.float32), np.array([0, 1])),
                           add_noise(np.zeros(n_filters, dtype=np.float32), np.array([0, 1])),
                           add_noise(np.zeros(n_filters, dtype=np.float32), np.array([0, 1])),
                           add_noise(np.ones(n_filters, dtype=np.float32), np.array([0, 1]))]
            new_bn_layer.set_weights(new_weights)    
#basic operation
    def _insert_new_layers(self, new_layers, output_id):
        new_node_id = self._add_node(deepcopy(self.node_list[self.adj_list[output_id][0][0]]))
        temp_output_id = new_node_id
        for layer in new_layers[:-1]:
            temp_output_id = self.add_layer(layer, temp_output_id)

        self._add_edge(new_layers[-1], temp_output_id, self.adj_list[output_id][0][0])
        new_layers[-1].input = self.node_list[temp_output_id]
        new_layers[-1].output = self.node_list[self.adj_list[output_id][0][0]]
        self._redirect_edge(output_id, self.adj_list[output_id][0][0], new_node_id)

    def _block_end_node(self, layer_id, block_size):
        ret = self.layer_id_to_output_node_ids[layer_id][0]
        for i in range(block_size - 2):
            ret = self.adj_list[ret][0][0]
        return ret

    def _dense_block_end_node(self, layer_id):
        return self._block_end_node(layer_id, Constant.DENSE_BLOCK_DISTANCE)

    def _conv_block_end_node(self, layer_id):
        """Get the input node ID of the last layer in the block by layer ID.
            Return the input node ID of the last layer in the convolutional block.

        Args:
            layer_id: the convolutional layer ID.
        """
        return self._block_end_node(layer_id, Constant.CONV_BLOCK_DISTANCE)